In [1]:
!pip install telepot
!pip install geocoder

In [2]:
import numpy as np
import cv2
from keras.models import load_model
from collections import deque
import telepot
import time
import geocoder
import socket

In [3]:
def get_current_location():
    # Get current location
    location = geocoder.ip('me')
    location_info = f"LOCATION: {location.city}, {location.state}, {location.country}\n"
    return location_info

def print_results(video):
    # Initializing variables
    trueCount = 0
    imageSaved = 0
    filename = 'savedImage.jpg'
    video_filename = 'recordedVideo.avi'
    sendAlert = 0

    # Get current location
    location_info = get_current_location()

    # Loading pre-trained Keras model
    print("Loading model ...")
    model = load_model('./modelnew.h5')
    Q = deque(maxlen=128)

    # Capturing video
    vs = cv2.VideoCapture(video)
    writer = None
    (W, H) = (None, None)
    count = 0
    
    # Looping through video frames
    while True:
        (grabbed, frame) = vs.read()

        if not grabbed:
            break

        if W is None or H is None:
            (H, W) = frame.shape[:2]

        output = frame.copy()

        # Preprocessing frame for model input
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (128, 128)).astype("float32")
        frame = frame.reshape(128, 128, 3) / 255

        # Predicting violence probability
        preds = model.predict(np.expand_dims(frame, axis=0))[0]
        Q.append(preds)

        # Calculating average prediction
        results = np.array(Q).mean(axis=0)
        i = (preds > 0.50)[0]  # Thresholding predictions
        label = i

        # Determining text color based on violence label
        text_color = (0, 255, 0)  # default: green
        if label:  # Violence prob
            text_color = (0, 0, 255)  # red
            trueCount += 1

        # Adding text to the output frame
        text = "Violence: {}".format(label)
        FONT = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(output, text, (35, 50), FONT, 1.25, text_color, 3)

        # Initializing video writer if not done
        if writer is None:
            fourcc = cv2.VideoWriter_fourcc(*"MJPG")
            writer = cv2.VideoWriter(video_filename, fourcc, 30, (W, H), True)

        # Writing frame to output video
        writer.write(output)

        # Displaying output frame
        cv2.imshow("Output", output)

        # Saving image and sending alert if violence detected for 50 consecutive frames
        if trueCount == 50:
            if imageSaved == 0:
                if label:
                    cv2.imwrite(filename, output)  # Save the frame
                    imageSaved = 1

            if sendAlert == 0:
                timeMoment = time.strftime("%H:%M:%S \nDate %d-%m-%y")
                bot = telepot.Bot('6457674371:AAE04wSEIjA4pyEvlYeYgqNQ7H0WgRuTiBg')
                bot.sendMessage(-1002037531635, f"VIOLENCE ALERT!!\n{location_info}TIME: {timeMoment}")
                with open(video_filename, 'rb') as video_file:
                    bot.sendVideo(-1002037531635, video_file)
                bot.sendPhoto(-1002037531635, photo=open(filename, 'rb'))
                sendAlert = 1

        # Checking for key press to quit
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

    # Cleaning up
    print("[INFO] cleaning up...")
    writer.release()
    vs.release()

In [6]:
#violence
V_path = "./New Dataset/Fight-20240507T205137Z-001/Fight/F1.4.mp4"
print_results(V_path)

Loading model ...


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━

In [5]:
#Non-violence
NV_path = "./New Dataset/Non Fight-20240507T205137Z-001/Non Fight/NF1.7.mp4"
print_results(NV_path)

Loading model ...


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━